# Holdout TS

一般的に使われているTarget Encodingの手法

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
#生成した特徴量を入れるseriesオブジェクトを用意しておく
ts = pd.Series(np.empty(df.shape[0]), index=df.index)

#カテゴリー変数ごとの目的変数の合計と数をカウントしておく
male_df = df.groupby('user_id').agg({'chese': ['sum', 'count']})

for _, holdout_idx in folds.split(df, df.chese):
    # ホールドアウトする行を取り出す
    holdout_df = df.iloc[holdout_idx]
    
    # ホールドアウトしたデータで合計とカウントを計算する
    holdout_1_df = holdout_df.groupby('user_id').agg({'chese': ['sum', 'count']})
    
    # 全体の集計からホールドアウトした分を引く
    train_1_df = male_df - holdout_1_df
    
    # ホールドアウトしたデータの平均値を計算していく
    oof_ts = holdout_df.apply(lambda row: train_1_df.loc[row.user_id][('chese', 'sum')] \
                                           / (train_1_df.loc[row.user_id][('chese', 'count')] + 1), axis=1)
    # 生成した特徴量を記録する
    ts[oof_ts.index] = oof_ts
    
ts.name = 'target_user_id'
df = df.join(ts)